In [3]:
from simple_pb2 import MyObj, Foo
obj=MyObj(name='hello')
print(obj)

message = Foo()
message.name = "Bender"
assert message.HasField("name")
message.serial_number = 2716057
assert message.HasField("serial_number")
assert not message.HasField("name")

name: "hello"



In [9]:
message.ClearField("test_oneof")

assert message.WhichOneof("test_oneof") is None
message.name = "Bender"
assert message.WhichOneof("test_oneof") == "name"

# HasField and ClearField also accept oneof names in addition 
# to field names:

# assert not message.HasField("test_oneof")
message.name = "Bender"
# assert message.HasField("test_oneof")
message.serial_number = 2716057
# assert message.HasField("test_oneof")
message.ClearField("test_oneof")
assert not message.HasField("test_oneof")
assert not message.HasField("serial_number")

In [2]:
from protocol.nlpserv_pb2 import PingRequest, NlParseRequest
ping=PingRequest(name='you')
print(ping)
req=NlParseRequest(text="徐先生还具体帮助他确定了把画雄鹰、松鼠和麻雀作为主攻目标。")
print(req)

name: "you"

text: "\345\276\220\345\205\210\347\224\237\350\277\230\345\205\267\344\275\223\345\270\256\345\212\251\344\273\226\347\241\256\345\256\232\344\272\206\346\212\212\347\224\273\351\233\204\351\271\260\343\200\201\346\235\276\351\274\240\345\222\214\351\272\273\351\233\200\344\275\234\344\270\272\344\270\273\346\224\273\347\233\256\346\240\207\343\200\202"



In [2]:
from concurrent import futures
import time

import grpc
import nlpserv_pb2
import nlpserv_pb2_grpc

def nlp_procs(p, func):
    # NOTE(gRPC Python Team): .close() is possible on a channel and should be
    # used in circumstances in which the with statement does not fit the needs
    # of the code.
    # with grpc.insecure_channel('localhost:10052') as channel:
    with grpc.insecure_channel(
            target='localhost:10052',
            options=[('grpc.lb_policy_name', 'pick_first'),
                     ('grpc.enable_retries', 0), ('grpc.keepalive_timeout_ms',
                                                  10000)]) as channel:
        stub = nlpserv_pb2_grpc.NlpProcsStub(channel)
        response = func(stub, p)
    # print("Greeter client received: " + response.message)
    return response

def get_head_lemma(tree, id):
    if id==0:
        return "_core_"
    for word in result.words:
        if word.id==id:
            return word.lemma

text="徐先生还具体帮助他确定了把画雄鹰、松鼠和麻雀作为主攻目标。"
result=nlp_procs(text, lambda stub, s: stub.ParseDependency(nlpserv_pb2.NlParseRequest(text=s)))   
for word in result.words:
    print(word.id, word.lemma, word.deprel, word.head_id, \
          get_head_lemma(result, word.head_id))

1 徐先生 主谓关系 4 帮助
2 还 状中结构 4 帮助
3 具体 状中结构 4 帮助
4 帮助 核心关系 0 _core_
5 他 兼语 4 帮助
6 确定 动宾关系 4 帮助
7 了 右附加关系 6 确定
8 把 状中结构 15 作为
9 画 介宾关系 8 把
10 雄鹰 动宾关系 9 画
11 、 标点符号 12 松鼠
12 松鼠 并列关系 10 雄鹰
13 和 左附加关系 14 麻雀
14 麻雀 并列关系 10 雄鹰
15 作为 动宾关系 6 确定
16 主攻 定中关系 17 目标
17 目标 动宾关系 15 作为
18 。 标点符号 4 帮助


In [5]:
add = lambda x, y: x + y
print(add(3, 5))

def invoker(func):
   return func(3, 6)
result=invoker(lambda x, y: x + y)
print(result)

8
9


In [5]:
result=nlp_procs(text, lambda stub, s: stub.GetPinyin(nlpserv_pb2.NlPinyinRequest(text=s, presentation=1)))   
print(result.text)

xú xiān shēng huán jù tǐ bāng zhù tā què dìng le bǎ huà xióng yīng none sōng shǔ hé má què zuò wéi zhǔ gōng mù biāo none 


In [3]:
docs=[
    "山东苹果丰收",
    "农民在江苏种水稻",
    "奥运会女排夺冠",
    "世界锦标赛胜出",
    "中国足球失败"
]
p=nlpserv_pb2.NlDocumentSet(textList=docs)
nlp_procs(p, lambda stub, p: stub.AddDocuments(p))

In [4]:
p=nlpserv_pb2.NlText(text="体育")
result=nlp_procs(p, lambda stub, p: stub.GetNearestDocuments(p))
for doc in result.docs:
    print(doc.content, doc.similarity)

中国足球失败 0.3437924385070801
奥运会女排夺冠 0.2620129883289337
世界锦标赛胜出 0.18041817843914032
农民在江苏种水稻 0.1255408078432083
山东苹果丰收 0.08718200027942657


In [6]:
import nlpserv_pb2 as nlp_messages
import nlpserv_pb2_grpc as nlp_service
p = nlp_messages.NlTokenizerRequest(text=nlp_messages.NlText(text="这里是北京"))
# response = client.Tokenizer(request)
result=nlp_procs(p, lambda stub, p: stub.Tokenizer(p))
for resp in result.tokens:
    print(resp.text)

这里
是
北京
